# Olympic Project - road to gold

## Questions:
 1. How has the performance of athletes changed based on gender, and has this led to a reduction in the performance gap?
 2. Can past Olympic results reliably predict future outcomes?

## Hypothesis:


# Olympic performance


In [62]:
#import of the data frame
from selenium import webdriver 
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import re


In [ ]:
#setting the first page
driver = webdriver.Chrome()
driver.get('https://olympics.com/en/olympic-games/beijing-2022/results/alpine-skiing/men-alpine-combined')
cookies_button = driver.find_element('css selector','#onetrust-accept-btn-handler')
cookies_button.click()
time.sleep(2)

In [ ]:
#select the Olympic Game
button_games = driver.find_element('css selector','button[data-cy=game-select]')
button_games.click()
all_games = driver.find_element('css selector','div [data-cy="game-list-wrapper"]')
first_game = all_games.find_elements('css selector','button [data-cy]')[4]
first_game.click()

In [ ]:
#Select the dicipline in the Olympic Game selected
button_discipline = driver.find_element('css selector', 'button[data-cy=discipline-select]')
button_discipline.click()
all_disciplines = driver.find_element('css selector','section [data-cy="disciplines-list"]')
first_discipline = all_disciplines.find_elements('css selector','button')[6]
first_discipline.click()

In [ ]:
#select the event in the dicipline selected
button_event = driver.find_element('css selector','button[data-cy=event-select]')
button_event.click()
event = driver.find_elements('css selector','button[data-cy=event-button]')[2]
event.click()

In [ ]:
#Go to the result of the event selected
button_go = driver.find_element('css selector','a[data-cy=go-link]')
button_go.click()

# Getting the imformation

In [ ]:
def info_extraction(soup):
    ''' to complete'''
    #location and indentification of the required information
    table = soup.find('div',attrs={'data-cy':'table-content'})
    rows = table.find_all('div',attrs={'data-row-id':True})
    #list of the info to capture
    countries = []
    participant=[]
    results=[]
    #loop to extract all events information of the dicipline selected
    for row in rows:
        try:
            countries.append(row.find('span',attrs={'class':'styles__CountryName-sc-1r5phm6-1 eQULfE'}).text)
        except:
            countries.append(None)
        try:
            participant.append(row.find('h3',attrs={'data-cy':'athlete-name'}).text)
        except:
            participant.append(None)
        try:
            results.append(row.find('span',attrs={'data-cy':'result-info-content'}).text)
        except:
            results.append(None)
    info_event=pd.DataFrame({'country':countries,'participant':participant,'result':results})
    #add a column with the olympics games name
    olympicg= soup.find('button',attrs={'data-cy':'game-select'}).text
    info_event['olympic_game']=olympicg
    #add a column with the dicipline name
    discipline=soup.find('button',attrs={'data-cy':'discipline-select'}).text
    info_event['discipline']=discipline
    #add a column with the event name
    event=soup.find('button',attrs={'data-cy':'event-select'}).text
    info_event['event']=event
    #return the entire dataset with all the required information
    return info_event

In [ ]:
def data_grab(url):
    ''' to complete'''
    #create the list with the required information
    final_result=[]
    #calculate the number of event in the dicipline selected
    driver.get(url)
    time.sleep(5)
    button_event = driver.find_element('css selector','button[data-cy=event-select]')
    button_event.click()
    events = len(driver.find_elements('css selector','button[data-cy=event-button]'))
    button_event.click()
    #loop to go though all the url and grab the required information or print the url where it did't work
    try:    
        for i in range(events):
            button_event = driver.find_element('css selector','button[data-cy=event-select]')
            button_event.click()
            eventfor=driver.find_elements('css selector','button[data-cy=event-button]')
            eventfor=eventfor[i]
            eventfor.click()
            #Go to the result of the event selected
            button_go = driver.find_element('css selector','a[data-cy=go-link]')
            button_go.click()
            time.sleep(5)
            soup= BeautifulSoup(driver.page_source)
            final_result.append(info_extraction(soup))
        pd.concat(final_result).to_csv(url.split('olympic-games/')[1].replace('/','_')+'.csv')
    except:
        print(url)

In [ ]:
url_list=pd.read_csv('olympics_url.csv').iloc[:,1].to_list()
driver = webdriver.Chrome()
driver.get('https://olympics.com/en/olympic-games/beijing-2022/results/alpine-skiing')
time.sleep(5)
cookies_button = driver.find_element('css selector','#onetrust-accept-btn-handler')
cookies_button.click()
time.sleep(5)
list(map(data_grab,url_list))

1. Connect (sucessfully) with the website - requests
2. Transforming the data from string to searchable object - make the soup!
3. Find out what data I need and locate it in the website -  soup.find_all or soup.select
4. Transform data to put it in a dataframe - pd.DataFrame
5. Create the spider to apply code for several pages - for loop

# Cleanning

In [110]:
#select the event with comparable result
csv_list = [pd.read_csv(fil) for fil in os.listdir() if fil.endswith('swimming.csv') or fil.endswith('weightlifting.csv') or fil.endswith('athletics.csv')
             or fil.endswith('cycling-track.csv') or fil.endswith('rowing.csv') or fil.endswith('sailing.csv')
             ]
data_olympic=pd.concat(csv_list)
data_olympic.drop(columns='Unnamed: 0',inplace=True)
#drop rows if result==nan
data_olympic.dropna(subset='result',inplace=True)
#divide olympic_game columns into two columns olympic_host and olympic_game_year
data_olympic['olympic_host']=data_olympic['olympic_game'].str.split(' ').str[0]
data_olympic['olympic_game_year']=data_olympic['olympic_game'].str.split(' ').str[1]
data_olympic.drop(columns='olympic_game',inplace=True)
#create and separete by gender using column event and drop rows of mix events
data_olympic['gender'] = data_olympic['event'].apply(lambda x: re.findall(r'\b(men|women)\b', x, flags=re.IGNORECASE)[0].lower() if re.findall(r'\b(men|women)\b', x, flags=re.IGNORECASE) else 'mix')
data_olympic=data_olympic[data_olympic['gender']!='mix']
#cleaning event
data_olympic['event'] = data_olympic['event'].apply(lambda x: re.sub(r'\b(?:men|women)\b', '', x, flags=re.IGNORECASE))
data_olympic['event'] = data_olympic['event'].apply(lambda x: re.sub(r'\b\'s\b', '', x, flags=re.IGNORECASE).strip().lower()) #!!!! this is not working

# Filtering

In [116]:
pd.pivot_table(data_olympic,index=['discipline','event'],columns='gender',values='result',aggfunc='count').sort_values(by='women',ascending=False).head(20) #this can be a function that give you this list or dict with x top events

gender                                            men  women
discipline    event                                         
Athletics     high jump                         268.0  220.0
Sailing       470 - two person dinghy           310.0  158.0
Athletics     long jump                         217.0  154.0
              discus throw                      209.0  135.0
              pentathlon                          NaN  128.0
              javelin throw                     178.0  118.0
              100m                              146.0  115.0
              shot put                          219.0  115.0
              200m                              211.0  112.0
Sailing       laser radial - one person dinghy    NaN  106.0
Swimming      100m freestyle                    117.0   94.0
Athletics     4x100m relay                      124.0   92.0
Swimming      100m backstroke                    98.0   83.0
Sailing       board mistral                       NaN   82.0
Cycling Track points race                       182.0   81.0
Sailing       singlehanded dinghy europe          NaN   76.0
Swimming      200m breaststroke                 108.0   76.0
              50m freestyle                      81.0   75.0
Athletics     800m                              154.0   73.0
              80m hurdles                         NaN   67.0

# Creating comparable tables

In [156]:
#for men
high_jump=data_olympic[data_olympic['event']=='high jump']
high_jump_men=high_jump[high_jump['gender']=='men']
high_jump_mens= pd.pivot_table(high_jump_men, index=['olympic_game_year', 'participant'], columns='gender', values='result', aggfunc='sum').sort_values(by=['olympic_game_year', 'men'], ascending=[True, False])
high_jump_mens.reset_index(level='participant', inplace=True)
high_jump_mens.drop(columns='participant',inplace=True)
#for women
high_jump=data_olympic[data_olympic['event']=='high jump']
high_jump_women=high_jump[high_jump['gender']=='women']
high_jump_womens= pd.pivot_table(high_jump_women, index=['olympic_game_year', 'participant'], columns='gender', values='result', aggfunc='sum').sort_values(by=['olympic_game_year', 'women'], ascending=[True, False])
high_jump_womens.reset_index(level='participant', inplace=True)
high_jump_womens.drop(columns='participant',inplace=True)
#menge


In [183]:
test_1=pd.pivot_table(high_jump, index=['olympic_game_year', 'participant'], columns='gender', values='result', aggfunc='sum')
test_1.reset_index(level='participant', inplace=True)
test_1.drop(columns='participant',inplace=True)
test_1.fillna(0, inplace=True)
test_1.sort_values(by=['olympic_game_year','men','women'], ascending=[True,False,False]).head(60)

gender,men,women
olympic_game_year,,
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0
1928,Did not start,0


In [153]:
high_jump_mens

gender,men
olympic_game_year,
1928,Did not start
1928,Did not start
1928,Did not start
1928,Did not start
1928,Did not start
...,...
City,2.09
City,2.09
City,2.09


In [ ]:

WTF=pd.pivot_table(data_olympic,index=['discipline','event'],values='result',aggfunc='count').sort_values(by='result',ascending=False)

In [ ]:
olympic_swimming = [pd.read_csv(fil) for fil in os.listdir() if fil.endswith('swimming.csv')]
ds_olympic=pd.concat(olympic_swimming)

In [ ]:
ds_nonan=ds_olympic.dropna(subset='result')
ds_nonan

In [ ]:
pd.pivot_table(ds_nonan,index='event',values='result',aggfunc='count').sort_values(by='result',ascending=False).head(20)


In [ ]:
swimming_100.info()

In [ ]:
swimming_100=data_olympic[(data_olympic['event']== '100m freestyle men') | (data_olympic['event']== '100m freestyle women')]

In [ ]:
data_olympic[(data_olympic['event']== '100m freestyle men') | (data_olympic['event']== '100m freestyle women')]

In [ ]:
Weightlifting_data=data_olympic[data_olympic['discipline']=='Weightlifting']

In [ ]:
data_olympic['discipline']=='athletics'

In [ ]:
pd.pivot_table(data_olympic,index=['discipline','event'],values='result',aggfunc='count').sort_index(ascending=False).head(60)

In [ ]:
type(csv_list)